In [1]:
import numpy as np
import librosa
import glob
import os
from random import randint
import torch
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline


import import_ipynb
from tsn_dataloader import *

importing Jupyter notebook from tsn_dataloader.ipynb


In [4]:
class SEN_classify(nn.Module):
    def __init__(self):
        super().__init__()
        self.siamese_cnn = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=[4,128],stride=[1,128]),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, kernel_size=[4,1],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, kernel_size=[4,1],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            )
        
        self.fcWithDropout = nn.Sequential(
            # many weights in state_dict
            nn.Linear(256*336, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 2),
            nn.Softmax(dim=1),
            )
        
    def forward_once(self,x):
        out1 = self.siamese_cnn(x)
        batchSize = list(out1.size())[0]
        out = out1.view(batchSize,-1)
        predictions = self.fcWithDropout(out)
        return predictions
    
    def forward(self,x1,x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1,out2

In [5]:
def test_SEN_classify():
    x1 = torch.rand(10,1,345,128)
    x2 = torch.rand(10,1,345,128)
    model = SEN_classify()
    pre1,pre2= model(x1,x2)
    print(pre1.shape)
    print(pre1)
    print(pre2.shape)
    print(pre2)
    return pre1
    
# target = test_SEN_classify()

torch.Size([10, 5])
tensor([[-0.3307, -0.3833,  0.2020, -0.1937,  0.2390],
        [ 0.2255, -0.6276, -0.1628,  0.1090, -0.0026],
        [ 0.0897, -0.1879, -0.1939,  0.0362, -0.2939],
        [-0.4174, -0.3084, -0.0079, -0.3383, -0.1640],
        [ 0.2330, -0.4543, -0.1598,  0.2717, -0.2261],
        [ 0.3169, -0.3206, -0.3301, -0.1006, -0.1261],
        [-0.2272,  0.0352,  0.0202, -0.1403,  0.3511],
        [-0.5567,  0.0804, -0.2010,  0.4911,  0.2181],
        [ 0.2339, -0.2440, -0.0989,  0.7181,  0.2980],
        [ 0.1926, -0.0293,  0.0594, -0.1097,  0.3536]],
       grad_fn=<ThAddmmBackward>)
torch.Size([10, 5])
tensor([[ 0.5494, -0.1260,  0.1393,  0.4621, -0.3370],
        [-0.5267, -0.1867,  0.0143,  0.0461, -0.0301],
        [-0.0428, -0.3549, -0.1722,  0.1338, -0.1906],
        [ 0.4127, -0.1264, -0.0057, -0.1803,  0.3873],
        [ 0.3357,  0.1785, -0.0948, -0.2974,  0.5829],
        [-0.2462, -0.2879,  0.1018,  0.1665, -0.0429],
        [ 0.0347, -0.5418, -0.1877, -0.1419, 